In [1]:
import numpy as np
import baostock as bs
import pandas as pd
from numpy import  linalg
def blacklitterman(returns,tau,P,Q):
    mu = returns.mean()
    sigma = returns.cov()
    pil = np.expand_dims(mu,axis = 0).T
    ts = tau * sigma
    ts_1 = linalg.inv(ts)
    Omega = np.dot(np.dot(P,ts), P.T)* np.eye(Q.shape[0])
    Omega_1 = linalg.inv(Omega)
    er = np.dot(linalg.inv(ts_1 + np.dot(np.dot(P.T,Omega_1),P)),(np.dot(ts_1 ,pil)+np.dot(np.dot(P.T,Omega_1),Q)))
    posterirorSigma = linalg.inv(ts_1 + np.dot(np.dot(P.T,Omega_1),P))
    return [er, posterirorSigma]

In [2]:
pick1 = np.array([1,0,1,1,1])
q1 = np.array([0.003*4])
pick2 = np.array([0.5,0.5,0,0,-1])
q2 = np.array([0.001])
P = np.array([pick1,pick2])
Q = np.array([q1,q2])

In [5]:
def get_stock_data(t1,t2,stock_name):
    lg = bs.login()
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)

    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节
    rs = bs.query_history_k_data(stock_name,
                                 "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
                                 start_date=t1, end_date=t2,
                                 frequency="d", adjustflag="3")
    print('query_history_k_data respond error_code:' + rs.error_code)
    print('query_history_k_data respond  error_msg:' + rs.error_msg)

    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    print(result)

    #### 结果集输出到csv文件 ####
    result.to_csv("D:\Job seeking\Huatai International\Week 2\history_A_stock_k_data.csv", index=False)
    print(result)

    #### 登出系统 ####
    bs.logout()
    result['date'] = pd.to_datetime(result['date'])
    result.set_index("date", inplace=True)
    return result

byjc = get_stock_data('2014-1-1','2015-1-1','sh.600004')
hxyh = get_stock_data('2014-1-1','2015-1-1','sh.600015')
zndl = get_stock_data('2014-1-1','2015-1-1','sh.600023')
fjgs = get_stock_data('2014-1-1','2015-1-1','sh.600033')
sykj = get_stock_data('2014-1-1','2015-1-1','sh.600183')


by = byjc['pctChg']
by.name = 'byjc'
by = pd.DataFrame(by,dtype=np.float)/100


hx = hxyh['pctChg']
hx.name = 'hxyh'
hx = pd.DataFrame(hx,dtype=np.float)/100

zn = zndl['pctChg']
zn.name = 'zndl'
zn = pd.DataFrame(zn,dtype=np.float)/100

fj = fjgs['pctChg']
fj.name = 'fjgs'
fj = pd.DataFrame(fj,dtype=np.float)/100

sy = sykj['pctChg']
sy.name = 'sykj'
sy = pd.DataFrame(sy,dtype=np.float)/100

sh_return = pd.concat([by,fj,hx,sy,zn],axis=1)
res = blacklitterman(sh_return,0.1,P,Q)
p_mean = pd.DataFrame(res[0],index = sh_return.columns, columns = ['posterior_mean'])
p_cov = res[1]
print(p_mean)
print(p_cov)


login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data respond error_code:0
query_history_k_data respond  error_msg:success
           date       code     open     high      low    close preclose  \
0    2014-01-02  sh.600004   6.9300   6.9700   6.9000   6.9400   6.9500   
1    2014-01-03  sh.600004   6.9300   6.9300   6.7700   6.8800   6.9400   
2    2014-01-06  sh.600004   6.8400   6.8400   6.7000   6.7500   6.8800   
3    2014-01-07  sh.600004   6.7100   6.7700   6.6600   6.7000   6.7500   
4    2014-01-08  sh.600004   6.7000   6.7600   6.6800   6.7400   6.7000   
..          ...        ...      ...      ...      ...      ...      ...   
240  2014-12-25  sh.600004  11.4000  11.7400  11.3600  11.6400  11.4200   
241  2014-12-26  sh.600004  11.6800  11.9200  11.5300  11.8300  11.6400   
242  2014-12-29  sh.600004  11.6400  11.6600  11.0900  11.2500  11.8300   
243  2014-12-30  sh.600004  11.1800  11.2600  10.6500  10.7800  11.2500   
244  2014-

C:\Users\79298\AppData\Local\Temp\ipykernel_20932\379161169.py:42: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  by = pd.DataFrame(by,dtype=np.float)/100
C:\Users\79298\AppData\Local\Temp\ipykernel_20932\379161169.py:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hx = pd.DataFrame(hx,dtype=np.float)/100
C:\Users\79298\AppData\Local\Tem

In [6]:
def blminVar(blres, goalRet):
    covs = np.array(blres[1],dtype=float)
    means = np.array(blres[0],dtype=float)
    L1 = np.append(np.append(covs.swapaxes(0,1),[means.flatten()],axis=0),
                   [np.ones(len(means))],axis=0).swapaxes(0,1)

    L2 = list(np.ones(len(means)))
    L2.extend([0,0])
    L3 = list(means)
    L3.extend([0,0])
    L4 = np.array([L2,L3],dtype=float)
    L = np.append(L1,L4,axis=0)
    results = linalg.solve(L,np.append(np.zeros(len(means)),[1,goalRet]))

    return pd.DataFrame(results[:-2],columns = ['p_weight'])

blresult = blminVar(res,0.70/252)
print(blresult)

   p_weight
0  0.246287
1  0.196282
2  0.254303
3  0.279113
4  0.024014


C:\Users\79298\AppData\Local\Temp\ipykernel_20932\155825299.py:11: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  L4 = np.array([L2,L3],dtype=float)
